In [2]:
from PIL import Image
import numpy as np
import os
import time


In [6]:
def image_to_np_array(image_name: str) -> np.array:
    img_src = Image.open(path.join('input', image_name)).convert('RGB')
    return np.array(img_src)

In [7]:
def to_semitone(img):
    return (0.3 * img[:, :, 0] + 0.59 * img[:, :, 1] + 0.11 *
            img[:, :, 2]).astype(np.uint8)

<!-- def semitone(img_arr):
    height = img_arr.shape[0]
    width = img_arr.shape[1]

    new_img_arr = np.zeros(shape=(height, width))

    for x in range(width):
        for y in range(height):
            pixel = old_img_arr[y, x]
            new_img_arr[y, x] = 0.3 * pixel[0] + 0.59 * pixel[1] + 0.11 * pixel[2]

    return new_img_arr.astype(np.uint8) -->

In [14]:
# loacl threshold for every pixel
def compute_local_threshold(image, window_size=15, C=10):
    
    height, width = image.shape
    padded_image = np.pad(image, pad_width=window_size // 2, mode='edge')
    threshold_image = np.zeros_like(image)

    for i in range(height):
        for j in range(width):
            local_area = padded_image[i:i + window_size, j:j + window_size]
            local_mean = np.mean(local_area)
            threshold_image[i, j] = local_mean - C

    return threshold_image

In [20]:
# method of WAN binarization
def wan_bin(image, window_size=15, C=8):
    if image.ndim == 3:
        image = to_semitone(image).astype(np.uint8)

    threshold_image = compute_local_threshold(image, window_size, C)
    binarized_image = np.where(image > threshold_image, 255, 0).astype(np.uint8)

    return binarized_image

In [21]:
def exec_img(img_name):
    time_start = time.time()

    img_src = Image.open('input/' + img_name).convert('RGB')

    gray_img_arr = to_semitone(np.array(img_src))  

    bin_img_arr = wan_bin(gray_img_arr)

    img = Image.fromarray(bin_img_arr)
    img.save("output/semitoned_" + img_name + ".png")
    print("time on {} : {}s".format(img_name, time.time() - time_start))


In [24]:
def main():
    images = os.listdir("input/")
    # images = ["face.png", "im1.png"]
    time_start = time.time()
    for i in range(len(images)):
        exec_img(images[i])
    print("     total time on {} images : {}s".format( len(images) ,time.time() - time_start))

if __name__ == "__main__":
    main()

time on 84_3.png : 86.26634955406189s
time on 198_115.png : 122.91692662239075s
time on face.png : 14.30328369140625s
time on im1.png : 15.972792863845825s
     total time on 4 images : 239.46655440330505s
